In [7]:
from __future__ import print_function
import pandas as pd
import primer3
from intermine.webservice import Service
service = Service("https://phytozome.jgi.doe.gov/phytomine/service")

def get_cds(name): # name should be transcript name
    template = service.get_template('Transcript_CDS_sequence')
    rows = template.rows(
        A = {"op": "=", "value": name}
        )
    for row in rows:
        return row["CDSs.sequence.residues"]



In [5]:
def get_primers_as_df(gene_list,linkerF,linkerR): # transcript name
    oligo_calc = primer3.thermoanalysis.ThermoAnalysis(mv_conc=50, dv_conc=1.5, dntp_conc=0.6, dna_conc=50, max_nn_length=60)
    #PRIMER_SALT_MONOVALENT = 50
    #PRIMER_SALT_DIVALENT  = 1.5
    #PRIMER_DNTP_CONC = 0.6
    #PRIMER_DNA_CONC = 50
    # nn = nearest neighbor (NN) thermodynamics
    
    
    dic_result = {
        'Transcript name' : [],
        'Primer L' : [],
        'Primer R' : [],
        'Primer L wl' : [],
        'Primer R wl' : [],
        
       # 'Primer L w/ linker' : [],
        'Tm L' : [],
        'Tm R' : [],
        'Tm L wl' :[],
        'Tm R wl' :[],
        
        'Length' : [],
        'Seq' : []
    }
    for gene in gene_list:
        seq = str(get_cds(gene)) # transcript name 
        #print (gene,seq)
        c = primer3.bindings.designPrimers(
            {
                'SEQUENCE_ID': 'temp',
                'SEQUENCE_TEMPLATE': seq,
            },
            {
                'PRIMER_OPT_SIZE': 20,
                'PRIMER_PICK_ANYWAY':1,
                'PRIMER_MIN_SIZE': 18,
                'PRIMER_MAX_SIZE': 25,
                'PRIMER_TASK' : 'pick_cloning_primers'

            })
        #print(c['PRIMER_LEFT_0_SEQUENCE'])
        left_list = list(set([c['PRIMER_LEFT_0_SEQUENCE'],c['PRIMER_LEFT_1_SEQUENCE'],c['PRIMER_LEFT_2_SEQUENCE'],c['PRIMER_LEFT_3_SEQUENCE'],c['PRIMER_LEFT_4_SEQUENCE']]))
        left_list_wl = [linkerF+x for x in left_list]
        right_list = list(set([c['PRIMER_RIGHT_0_SEQUENCE'],c['PRIMER_RIGHT_1_SEQUENCE'],c['PRIMER_RIGHT_2_SEQUENCE'],c['PRIMER_RIGHT_3_SEQUENCE'],c['PRIMER_RIGHT_4_SEQUENCE']]))
        right_list_wl = [linkerR+x for x in right_list]
        dic_result['Transcript name'].append(gene)
        
        
        dic_result['Primer L'].append(left_list)
        dic_result['Primer R'].append(right_list)
        dic_result['Primer L wl'].append(left_list_wl)
        dic_result['Primer R wl'].append(right_list_wl)
        
        dic_result['Tm L'].append([oligo_calc.calcTm(x) for x in left_list])
        dic_result['Tm R'].append([oligo_calc.calcTm(x) for x in right_list])
        dic_result['Tm L wl'].append([oligo_calc.calcTm(x) for x in left_list_wl])
        dic_result['Tm R wl'].append([oligo_calc.calcTm(x) for x in right_list_wl])
        
        dic_result['Length'].append(len(seq))
        dic_result['Seq'].append(seq)
    return pd.DataFrame(dic_result)



---
## 밑으로는 다 실행 과정 및 결과들##
---

In [119]:
# 입력해서 만들기
genelist = '''>AT2G16835.1 | Symbols:  | Aquaporin-like superfamily protein | chr2:7295851-7296318 FORWARD LENGTH=243
>AT1G52180.1 | Symbols:  | Aquaporin-like superfamily protein | chr1:19424944-19425928 REVERSE LENGTH=375
>AT1G15825.1 | Symbols:  | hydroxyproline-rich glycoprotein family protein | chr1:5448149-5448529 REVERSE LENGTH=381
>AT2G26040.1 | Symbols: PYL2, RCAR14 | PYR1-like 2 | chr2:11094832-11095404 REVERSE LENGTH=573
>AT1G02820.1 | Symbols:  | Late embryogenesis abundant 3 (LEA3) family protein | chr1:623765-624475 REVERSE LENGTH=615
>AT1G01360.1 | Symbols: RCAR1, PYL9 | regulatory component of ABA receptor 1 | chr1:141971-143183 FORWARD LENGTH=1000
>AT1G27730.1 | Symbols: STZ, ZAT10 | salt tolerance zinc finger | chr1:9648141-9649073 REVERSE LENGTH=933
>AT1G28490.1 | Symbols: SYP61, ATSYP61, OSM1 | syntaxin of plants 61 | chr1:10015919-10018047 FORWARD LENGTH=1076
>AT1G45249.3 | Symbols: ABF2 | abscisic acid responsive elements-binding factor 2 | chr1:17165178-17167907 REVERSE LENGTH=1697
>AT1G32640.1 | Symbols: ATMYC2, RD22BP1, JAI1, JIN1, MYC2, ZBF1 | Basic helix-loop-helix (bHLH) DNA-binding family protein | chr1:11798810-11800988 REVERSE LENGTH=2179
>AT1G71960.1 | Symbols: ABCG25, ATABCG25 | ATP-binding casette family G25 | chr1:27082394-27088248 REVERSE LENGTH=2267
>AT1G72770.1 | Symbols: HAB1 | homology to ABI1 | chr1:27390022-27393281 FORWARD LENGTH=2551
>AT1G73360.1 | Symbols: HDG11, EDT1, ATHDG11 | homeodomain GLABROUS 11 | chr1:27578696-27582521 REVERSE LENGTH=3067
>AT2G18960.1 | Symbols: AHA1, PMA, OST2, HA1 | H(+)-ATPase 1 | chr2:8221631-8227622 FORWARD LENGTH=3431
>AT1G15520.1 | Symbols: PDR12, ATPDR12, ABCG40, ATABCG40 | pleiotropic drug resistance 12 | chr1:5331858-5338326 REVERSE LENGTH=4558
>AT1G24190.1 | Symbols: SIN3, ATSIN3, SNL3 | SIN3-like 3 | chr1:8563558-8570505 REVERSE LENGTH=4441'''.split('\n')
genelist = [x .split('|')[0].strip().replace('>','') for x in genelist]
df_ath_test = get_primers_as_df(genelist,'GAATTCGGCCGTCAAGGCCAGAAGGAGATATAACC','AGTCGACGGCCCATGAGGCCGATATC')

In [120]:
df_ath_test.to_csv('DroughtNet.SFI1.Ath.test.txt',sep='\t')

---

In [7]:
# 파일에서 만들기 
file_in = './1.GeneToHomologCluster.out.txt'

df_in = pd.read_csv(file_in,sep='\t',header=None)

ATH_list = []
ALY_list = []
ESA_list = []
for i in df_in.index:
    ATH = df_in.loc[i][0]
    ATH_list.append(ATH)
    ALY = str(df_in.loc[i][1])
    ALY_list.append(ALY)
    ESA = df_in.loc[i][2]
    ESA_list.append(ESA)
    if df_in.loc[i][5] != 'None':
        ATH_list += map(str,df_in.loc[i][5].split(','))
    if df_in.loc[i][6] != 'None':
        ALY_list += map(str,df_in.loc[i][6].split(','))
    if df_in.loc[i][7] != 'None':
        ESA_list += map(str,df_in.loc[i][7].split(','))


In [86]:
testlist = ATH_list[0:10]
df_primers_ath = get_primers_as_df(testlist)

---

In [2]:
genelist = 'AT1G02450.1,AT5G06950.1,AT1G64280.1,AT1G32640.1,AT1G15750.1'.split(',')

In [10]:
df_ath_test = get_primers_as_df(genelist,'GGAGATATAACC','GGCCGATATC')

In [13]:
df_ath_test.to_csv('primers.txt',sep='\t')

In [75]:
seq = 'ATGTTCCTAGTAAACTGGTTCTATGGCGTTCTGAACTACCTGGGTCTTTACAACAAGAACGCGAAGATCCTCTTCCTGGGCCTTGACAATGCGGGCAAGACCACGCTGATGCACATGCTCAAGGACGATCGCGTTGTGCAGCACCAGCCTACGCAGTACCCGACCTCCGAGGAGATTCAATTGGGCGGTATCAACTTCAAAGCGTTCGATCTCGGCGGCCATGAAATTGCGCGCCGGGTGTGGAAGGACTACTACGCCAAGGTGGACGCTATCGTCTACCTGGTGGACTCGGCGGACCGCGAGCGCTTCCCGGAGGCGAAGAAGGAGCTTGACGGGCTTCTGGGCGATGACTCGCTCGGTACCGTTCCCTTCCTGGTGCTGGGCAACAAGATCGACATTCCCACCGCCGCTTCGGAGGACGAGCTCCGCAACTGGCTTGGCCTCACATACACAACCGGCAAGGGCAAGGTCAACCTGAGCGACCAGAAGATTCGTCCCGTGGAGGTTTTCATGTGCTCGGTGGTTAAGCGGATGGGCTACGGCGAGGGCTTCCGGTGGCTGAGCCAGTACATCAAG'

c = primer3.bindings.designPrimers(
    {
        'SEQUENCE_ID': 'MH1000',
        'SEQUENCE_TEMPLATE': seq,
    },
    {
        'PRIMER_OPT_SIZE': 20,
        'PRIMER_PICK_ANYWAY':1,
        'PRIMER_MIN_SIZE': 18,
        'PRIMER_MAX_SIZE': 25,
        'PRIMER_TASK' : 'pick_cloning_primers'

    })
c

{'PRIMER_INTERNAL_EXPLAIN': 'considered 5, unacceptable product size 5, ok 5',
 'PRIMER_INTERNAL_NUM_RETURNED': 0L,
 'PRIMER_LEFT_0': (0L, 22L),
 'PRIMER_LEFT_0_END_STABILITY': 3.01,
 'PRIMER_LEFT_0_GC_PERCENT': 36.36363636363637,
 'PRIMER_LEFT_0_HAIRPIN_TH': 0.0,
 'PRIMER_LEFT_0_PENALTY': 7.543291298567226,
 'PRIMER_LEFT_0_PROBLEMS': ' Temperature too low;',
 'PRIMER_LEFT_0_SELF_ANY_TH': 6.752833955456651,
 'PRIMER_LEFT_0_SELF_END_TH': 0.0,
 'PRIMER_LEFT_0_SEQUENCE': 'ATGTTCCTAGTAAACTGGTTCT',
 'PRIMER_LEFT_0_TM': 54.456708701432774,
 'PRIMER_LEFT_1': (0L, 21L),
 'PRIMER_LEFT_1_END_STABILITY': 3.62,
 'PRIMER_LEFT_1_GC_PERCENT': 38.095238095238095,
 'PRIMER_LEFT_1_HAIRPIN_TH': 0.0,
 'PRIMER_LEFT_1_PENALTY': 8.125033861838574,
 'PRIMER_LEFT_1_PROBLEMS': ' Temperature too low;',
 'PRIMER_LEFT_1_SELF_ANY_TH': 6.752833955456651,
 'PRIMER_LEFT_1_SELF_END_TH': 0.0,
 'PRIMER_LEFT_1_SEQUENCE': 'ATGTTCCTAGTAAACTGGTTC',
 'PRIMER_LEFT_1_TM': 52.874966138161426,
 'PRIMER_LEFT_2': (0L, 22L),
 'PRIME

In [100]:
c = primer3.bindings.designPrimers(
    {
        'SEQUENCE_ID': 'MH1000',
        'SEQUENCE_TEMPLATE': seq,
    },
    {
        'PRIMER_OPT_SIZE': 20,
        'PRIMER_PICK_ANYWAY':1,
        'PRIMER_MIN_SIZE': 18,
        'PRIMER_MAX_SIZE': 25,
        'PRIMER_TASK' : 'pick_cloning_primers',
        'PRIMER_THERMODYNAMIC_ALIGNMENT':1,
        'PRIMER_TM_FORMULA':1,
        'PRIMER_SALT_CORRECTIONS':1
        #'PRIMER_SALT_DIVALENT':0

    })
c


{'PRIMER_INTERNAL_EXPLAIN': 'considered 5, unacceptable product size 5, ok 5',
 'PRIMER_INTERNAL_NUM_RETURNED': 0L,
 'PRIMER_LEFT_0': (0L, 22L),
 'PRIMER_LEFT_0_END_STABILITY': 3.01,
 'PRIMER_LEFT_0_GC_PERCENT': 36.36363636363637,
 'PRIMER_LEFT_0_HAIRPIN_TH': 0.0,
 'PRIMER_LEFT_0_PENALTY': 7.543291298567226,
 'PRIMER_LEFT_0_PROBLEMS': ' Temperature too low;',
 'PRIMER_LEFT_0_SELF_ANY_TH': 6.752833955456651,
 'PRIMER_LEFT_0_SELF_END_TH': 0.0,
 'PRIMER_LEFT_0_SEQUENCE': 'ATGTTCCTAGTAAACTGGTTCT',
 'PRIMER_LEFT_0_TM': 54.456708701432774,
 'PRIMER_LEFT_1': (0L, 21L),
 'PRIMER_LEFT_1_END_STABILITY': 3.62,
 'PRIMER_LEFT_1_GC_PERCENT': 38.095238095238095,
 'PRIMER_LEFT_1_HAIRPIN_TH': 0.0,
 'PRIMER_LEFT_1_PENALTY': 8.125033861838574,
 'PRIMER_LEFT_1_PROBLEMS': ' Temperature too low;',
 'PRIMER_LEFT_1_SELF_ANY_TH': 6.752833955456651,
 'PRIMER_LEFT_1_SELF_END_TH': 0.0,
 'PRIMER_LEFT_1_SEQUENCE': 'ATGTTCCTAGTAAACTGGTTC',
 'PRIMER_LEFT_1_TM': 52.874966138161426,
 'PRIMER_LEFT_2': (0L, 22L),
 'PRIME

In [55]:
primer3.calcHairpin('CCCCCATCCGATCAGGGGG')

ThermoResult(structure_found=True, tm=34.15, dg=337.09, dh=-36300.00, ds=-118.13)

In [102]:
primer3.calcTm('ATGTTCCTAGTAAACTGGTTCT',mv_conc=50, dv_conc=1.5, dntp_conc=0.6, dna_conc=50, max_nn_length=60)

54.456708701432774

In [59]:
primer3.bindings.calcHeterodimer('ATGAACAAGAACAAGATTGATGT','TCAACGAGGAGAAATATGAGAT')

ThermoResult(structure_found=True, tm=-29.81, dg=8441.74, dh=-71600.00, ds=-258.07)

In [60]:
primer3.bindings.calcHomodimer('ATGAACAAGAACAAGATTGATGT')

ThermoResult(structure_found=True, tm=-2.60, dg=3158.73, dh=-98200.00, ds=-326.81)

In [11]:
ESA_genelist = '''Thhalv10013861m
Thhalv10016867m
Thhalv10023658m
Thhalv10025022m
Thhalv10013468m
Thhalv10007349m
Thhalv10003358m
Thhalv10009782m
Thhalv10022669m
Thhalv10022654m
Thhalv10020604m
Thhalv10006121m
Thhalv10023595m
Thhalv10017213m
Thhalv10012813m
Thhalv10007162m
Thhalv10011316m
Thhalv10020675m
Thhalv10010770m
Thhalv10013168m
Thhalv10010694m
Thhalv10001028m
Thhalv10017142m
Thhalv10021628m
Thhalv10014279m
Thhalv10011770m
Thhalv10019115m
Thhalv10022188m
Thhalv10008924m
Thhalv10021565m
Thhalv10014824m
Thhalv10011768m
Thhalv10019262m
Thhalv10005072m
Thhalv10006473m
Thhalv10001004m
Thhalv10006278m
Thhalv10021680m
Thhalv10000338m
Thhalv10021006m
Thhalv10010492m
Thhalv10016856m
Thhalv10000915m
Thhalv10013999m
Thhalv10005719m
Thhalv10017050m
Thhalv10024324m
Thhalv10016157m
Thhalv10015427m
Thhalv10004178m
Thhalv10022628m
Thhalv10015443m
Thhalv10022892m
Thhalv10019250m
Thhalv10016752m
Thhalv10012750m
Thhalv10026013m
Thhalv10014695m
Thhalv10004102m
Thhalv10022078m
Thhalv10013655m
Thhalv10010407m
Thhalv10026283m
Thhalv10026916m
Thhalv10026303m
Thhalv10013473m
Thhalv10007672m
Thhalv10019346m
Thhalv10027426m
Thhalv10012960m
Thhalv10026218m
Thhalv10004256m
Thhalv10007772m
Thhalv10007359m
Thhalv10025628m
Thhalv10020507m
Thhalv10012832m
Thhalv10020096m
Thhalv10017729m
Thhalv10019224m
Thhalv10005590m
Thhalv10019205m
Thhalv10015419m
Thhalv10022531m
Thhalv10027674m
Thhalv10011412m
Thhalv10009880m
Thhalv10025435m
Thhalv10011552m
Thhalv10009185m
Thhalv10000507m
Thhalv10000507m
Thhalv10028738m
Thhalv10025531m
Thhalv10021629m
Thhalv10023336m
Thhalv10020542m
Thhalv10007183m
Thhalv10004651m
Thhalv10026126m
Thhalv10021174m
Thhalv10014474m
Thhalv10014099m
Thhalv10008373m
Thhalv10002002m
Thhalv10022600m
Thhalv10028900m
Thhalv10004956m
Thhalv10019231m
Thhalv10018912m
Thhalv10010317m
Thhalv10022958m
Thhalv10024568m
Thhalv10018420m
Thhalv10010244m
Thhalv10013040m
Thhalv10016663m
Thhalv10013567m
Thhalv10025680m
Thhalv10014171m
Thhalv10022774m
Thhalv10004581m
Thhalv10000239m
Thhalv10004596m
Thhalv10018830m
Thhalv10000495m
Thhalv10001541m
Thhalv10026803m
Thhalv10008077m
Thhalv10021155m
Thhalv10025673m
Thhalv10021152m
Thhalv10008244m
Thhalv10014085m
Thhalv10016858m
Thhalv10004528m
Thhalv10008189m
Thhalv10028769m
Thhalv10016868m
Thhalv10000958m
Thhalv10014117m
Thhalv10025717m
Thhalv10004596m
Thhalv10006516m
Thhalv10026332m
Thhalv10028546m
Thhalv10001020m
Thhalv10026372m
Thhalv10011283m
Thhalv10011332m
Thhalv10019640m
Thhalv10015875m
Thhalv10013480m
Thhalv10022861m
Thhalv10013175m
Thhalv10008995m
Thhalv10010967m
Thhalv10023722m
Thhalv10017023m
Thhalv10008594m
Thhalv10009044m
Thhalv10017144m
Thhalv10003429m
Thhalv10007845m
Thhalv10010627m
Thhalv10018014m
Thhalv10000225m
Thhalv10019947m
Thhalv10006857m
Thhalv10016599m
Thhalv10022685m
Thhalv10007324m
Thhalv10018282m
Thhalv10011252m
Thhalv10018894m
Thhalv10029001m
Thhalv10005892m
Thhalv10012732m
Thhalv10024396m
Thhalv10001310m
Thhalv10008065m
Thhalv10028777m
Thhalv10028623m
Thhalv10013535m
Thhalv10013449m
Thhalv10009666m
Thhalv10012271m
Thhalv10024309m
Thhalv10020906m
Thhalv10014975m
Thhalv10003935m
Thhalv10011438m
Thhalv10022637m
Thhalv10011405m
Thhalv10025232m
Thhalv10010541m
Thhalv10021426m
Thhalv10000967m
Thhalv10004625m
Thhalv10016949m
Thhalv10019463m
Thhalv10014136m
Thhalv10014146m
Thhalv10013820m
Thhalv10025911m
Thhalv10028739m
Thhalv10025942m
Thhalv10007418m
Thhalv10008957m
Thhalv10011399m
Thhalv10023430m
Thhalv10014739m
Thhalv10007293m
Thhalv10018366m
Thhalv10005884m
Thhalv10023481m
Thhalv10007717m
Thhalv10021313m
Thhalv10012754m
Thhalv10016611m
Thhalv10024278m
Thhalv10004675m
Thhalv10008201m
Thhalv10010497m
Thhalv10018295m
Thhalv10000386m
Thhalv10010733m
Thhalv10014873m
Thhalv10009677m
Thhalv10009955m
Thhalv10003429m
Thhalv10027035m
Thhalv10013260m
Thhalv10000983m
Thhalv10010150m
Thhalv10003616m
Thhalv10005924m
Thhalv10004131m
Thhalv10021377m
Thhalv10004150m
Thhalv10025746m
Thhalv10004669m
Thhalv10025379m
Thhalv10001422m
Thhalv10004812m
Thhalv10024781m
Thhalv10013679m
Thhalv10013680m
Thhalv10028957m
Thhalv10000311m
Thhalv10001645m
Thhalv10023725m
Thhalv10008131m
Thhalv10002711m
Thhalv10000219m
Thhalv10011415m
Thhalv10022521m
Thhalv10022882m
Thhalv10014245m
Thhalv10005608m
Thhalv10022235m
Thhalv10001600m
Thhalv10004645m
Thhalv10000259m
Thhalv10010084m
Thhalv10018774m
Thhalv10014998m
Thhalv10013029m
Thhalv10007663m
Thhalv10013695m
Thhalv10006957m
Thhalv10003492m
Thhalv10026931m
Thhalv10018181m
Thhalv10006893m
Thhalv10000051m
Thhalv10020098m
Thhalv10009494m
Thhalv10010830m
Thhalv10012840m
Thhalv10008246m
Thhalv10015064m
Thhalv10021449m
Thhalv10014374m
Thhalv10020127m
Thhalv10012864m
Thhalv10024591m
Thhalv10013480m
Thhalv10008528m
Thhalv10006726m
Thhalv10009707m
Thhalv10004968m
Thhalv10028863m
Thhalv10000989m
Thhalv10008975m
Thhalv10000861m
Thhalv10028920m
Thhalv10009224m
Thhalv10025928m
Thhalv10004938m
Thhalv10014029m
Thhalv10025534m
Thhalv10025685m
Thhalv10025604m
Thhalv10021549m
Thhalv10027854m
Thhalv10010153m
Thhalv10008246m
Thhalv10011967m
Thhalv10019997m
Thhalv10020083m
Thhalv10024382m
Thhalv10009047m
Thhalv10025910m
Thhalv10006141m
Thhalv10001573m
Thhalv10008397m
Thhalv10017874m
Thhalv10010598m
Thhalv10025940m
Thhalv10024449m
Thhalv10025900m
Thhalv10004748m
Thhalv10010684m
Thhalv10007339m
Thhalv10021014m
Thhalv10013907m
Thhalv10017366m
Thhalv10010790m
Thhalv10010325m
Thhalv10015253m
Thhalv10004576m
Thhalv10006866m
Thhalv10010281m
Thhalv10013689m
Thhalv10023486m
Thhalv10004619m
Thhalv10011756m
Thhalv10001603m
Thhalv10012742m
Thhalv10028542m
Thhalv10028759m
Thhalv10008668m
Thhalv10018581m
Thhalv10004806m
Thhalv10026069m
Thhalv10024517m
Thhalv10008985m
Thhalv10014630m
Thhalv10011471m
Thhalv10004208m
Thhalv10016651m
Thhalv10018288m
Thhalv10023282m
Thhalv10014617m
Thhalv10004977m'''.split('\n')

In [12]:
df_ESA_primers = get_primers_as_df(ESA_genelist)

In [15]:
df_ESA_primers.to_csv('ESA_primer.txt',sep='\t')